In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron



In [40]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train["train"] = 1
test["train"] = 0

df = pd.concat([train, test], axis=0)

# df.corr()["Survived"]


df.loc[df.Sex == "male", "Sex"] = 0
df.loc[df.Sex == "female", "Sex"] = 1
df["Gender"] = df["Sex"]
df["Relations"] = df["SibSp"] + df["Parch"]
df.drop(["SibSp", "Parch", "Cabin", "Ticket", "Sex", "Name", "PassengerId"], axis=1, inplace=True)

mean_age = int(df.Age.mean())
df.loc[df.Age.isnull(), "Age"] = mean_age

df.loc[df.Embarked.isnull(), "Embarked"] = "S"

for elem in df.Pclass.unique():
    mean_fare = df.loc[df.Pclass == elem, "Fare"].mean()
    df.loc[df.Fare.isnull() & df.Pclass == elem, "Fare"] = mean_fare
    
df = pd.get_dummies(df, columns=["Embarked"])

train = df[df.train == 1].copy()
test = df[df.train == 0].copy()

train.drop("train", axis=1, inplace=True)
test.drop(["train", "Survived"], axis=1, inplace=True)

train_y = train["Survived"].values
train_x = train[train.columns.drop("Survived")].values

test_x = test.values

In [57]:
model = Perceptron(n_jobs=-1, n_iter=10)
model.fit(train_x, train_y)

output = model.predict(test_x).astype(int)

In [60]:
test = pd.read_csv("test.csv")
output_df = pd.DataFrame()
output_df["PassengerId"] = test.PassengerId
output_df["Survived"] = output
output_df.to_csv("perceptron.csv", index=False)